In [1]:
run accelerator.py

RadioButtons(description='Device:', options=('GPU Gen9', 'GPU Iris XE Max', 'CPU Xeon 6128', 'CPU Xeon 8153'),…

## producer consumer problem in SYCL
This code is a simple example of using SYCL (a C++ based heterogeneous programming framework) to implement a producer-consumer pattern for data processing on a GPU.

The producer function generates some data (a vector of integers) and writes it to a buffer, while the consumer function reads the data from the same buffer, processes it (in this case, just printing it out), and releases the buffer.

The main function creates a SYCL queue object with a GPU selector, initializes the buffer, and launches the producer and consumer functions in sequence. The gpu_selector selects the first available GPU device for executing the code.

The code uses the buffer class to allocate memory on the device, and the handler class to define the scope of memory access operations. The accessor object is used to get read or write access to the buffer data. The submit method is used to submit the producer and consumer kernels to the device and to wait for them to complete using events. Finally, the wait method is used to synchronize the host and device execution.








In [1]:
%%writefile lab/prodc.cpp
#include <CL/sycl.hpp>
#include <vector>
#include <iostream>

using namespace cl::sycl;

const int BUFFER_SIZE = 10;
const int NUM_ITERATIONS = 5;

// Producer function
void producer(queue& q, buffer<int, 1>& buffer, int iteration) {
    std::vector<int> data(BUFFER_SIZE);
    for (int i = 0; i < BUFFER_SIZE; ++i) {
        data[i] = i + (BUFFER_SIZE * iteration);
    }

    // Write data to buffer
    auto write_event = q.submit([&](handler& cgh) {
        auto accessor = buffer.get_access<access::mode::write>(cgh);
        cgh.copy(data.data(), accessor);
    });

    // Wait for write event to complete
    write_event.wait();

    // Print the produced data
    std::cout << "Iteration " << iteration << " - Produced data:\n";
    for (int i = 0; i < BUFFER_SIZE; ++i) {
        std::cout << data[i] << " ";
    }
    std::cout << std::endl;
}

// Consumer function
void consumer(queue& q, buffer<int, 1>& buffer, int iteration) {
    std::vector<int> data(BUFFER_SIZE);

    // Read data from buffer
    auto read_event = q.submit([&](handler& cgh) {
        auto accessor = buffer.get_access<access::mode::read>(cgh);
        cgh.copy(accessor, data.data());
    });

    // Wait for read event to complete
    read_event.wait();

    // Print the consumed data
    std::cout << "Iteration " << iteration << " - Consumed data:\n";
    for (int i = 0; i < BUFFER_SIZE; ++i) {
        std::cout << data[i] << " ";
    }
    std::cout << std::endl;
}

int main() {
    queue q{sycl::gpu_selector_v};

    // Create buffer
    buffer<int, 1> buffer{range<1>{BUFFER_SIZE}};

    // Launch producer and consumer functions for multiple iterations
    for (int i = 0; i < NUM_ITERATIONS; ++i) {
        producer(q, buffer, i);
        consumer(q, buffer, i);
        std::cout << std::endl;
    }

    return 0;
}


Overwriting lab/prodc.cpp


In [2]:
! chmod 755 q; chmod 755 run_simple111.sh;if [ -x "$(command -v qsub)" ]; then ./q run_simple111.sh; else ./run_simple111.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Selected Device is: 

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2283397.v-qsvr-1           ...ub-singleuser u182761         00:01:43 R jupyterhub     
2283452.v-qsvr-1           run_simple01.sh  u182761                0 Q batch          
2283474.v-qsvr-1           run_simple111.sh u182761                0 Q batch          

Waiting for Output ████████████████████████████████████████████████████████████

TimeOut 60 seconds: Job is still queued for execution, check for output file later (run_simple111.sh.o2283474)

 Done⬇

########################################################################
#      Date:           Sun 23 Apr 2023 03:35:01 PM PDT
#    Job ID:           2283474.v-qsvr-1.aidevcloud
#      User:           u182761
# Resources:           cput=75:00:00,neednodes=1:gen9:ppn=2,nodes=1:gen9:ppn=2,wallti

In [2]:
%%writefile lab/dphi.cpp
#include <CL/sycl.hpp>
#include <iostream>
#include <vector>

using namespace sycl;

constexpr size_t num_philosophers = 5;

//************************************
// Simulate eating action for philosophers in parallel.
//************************************
void philosophers_eating(queue& q, size_t num_iterations) {
    // Range object for the number of philosophers.
    range<1> num_items{ num_philosophers };

    // Create a buffer that holds the philosopher IDs.
    std::vector<size_t> ids(num_philosophers);
    for (size_t i = 0; i < num_philosophers; ++i) {
        ids[i] = i;
    }
    buffer id_buf(ids);

    for (size_t i = 0; i < num_iterations; ++i) {
        q.submit([&](handler& h) {
            // Create an accessor with read_only access permission.
            accessor<size_t, 1, access::mode::read_write, access::target::global_buffer> ids(id_buf, h);

            // Use parallel_for to simulate eating action in parallel.
            h.parallel_for(num_items, [=](auto i) {
                ids[i] = i;
            });
        });

        // Wait for the tasks to complete.
        q.wait();

        // Print the messages on the host side.
        for (const auto& id : ids) {
            std::cout << "Philosopher " << id << " is eating iteration " << i << std::endl;
        }
    }
}

//************************************
// Main function.
//************************************
int main() {
    // Create device selector for the device of your interest.
    default_selector d_selector;

    try {
        queue q(d_selector);
        // Print out the device information used for the kernel code.
        std::cout << "Running on device: "
            << q.get_device().get_info<info::device::name>() << "\n";

        size_t num_iterations = 10;
        philosophers_eating(q, num_iterations);
    }
    catch (exception const& e) {
        std::cout << "An exception is caught for philosophers_eating.\n";
        std::terminate();
    }

    std::cout << "Philosophers eating simulation successfully completed on device.\n";
    return 0;
}


Overwriting lab/dphi.cpp


In [3]:
! chmod 755 q; chmod 755 run_simple01.sh;if [ -x "$(command -v qsub)" ]; then ./q run_simple01.sh; else ./run_simple01.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Selected Device is: 

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2283766.v-qsvr-1           ...ub-singleuser u182761         00:00:03 R jupyterhub     
2283767.v-qsvr-1           run_simple01.sh  u182761                0 Q batch          

Waiting for Output ███████████████████████████████ Done⬇

########################################################################
#      Date:           Mon 24 Apr 2023 02:37:12 AM PDT
#    Job ID:           2283767.v-qsvr-1.aidevcloud
#      User:           u182761
# Resources:           cput=75:00:00,neednodes=1:gen9:ppn=2,nodes=1:gen9:ppn=2,walltime=06:00:00
########################################################################

## u182761 is compiling SYCL Code -- oneAPI Hack2Skill example - 1 of 1 dphi.cpp
Running on device: Intel(R) UHD Graphics P630 [0x3e96]
Philosop

In [7]:

%%writefile lab/hellw.cpp
#include <CL/sycl.hpp>
#include <iostream>

int main() {
  // Create a SYCL queue
  sycl::queue myQueue;

  // Create a buffer for the message
  char message[] = "Hello, world!";
  size_t message_size = sizeof(message) / sizeof(char);
  sycl::buffer<char, 1> message_buffer(message, sycl::range<1>(message_size));

  // Define a command group to execute
  myQueue.submit([&](sycl::handler& myHandler) {
    // Get an accessor to the buffer
    auto message_accessor = message_buffer.get_access<sycl::access::mode::read>(myHandler);

    myHandler.single_task([=]() {
      // Print "Hello, world!" on the device
      for (size_t i = 0; i < message_size; ++i) {
        sycl::ext::oneapi::experimental::printf("%c", message_accessor[i]);
      }
    });
  });

  // Wait for the queue to finish executing
  myQueue.wait();

  return 0;
}


Overwriting lab/hellw.cpp


In [8]:
! chmod 755 q; chmod 755 run_simple02.sh;if [ -x "$(command -v qsub)" ]; then ./q run_simple02.sh; else ./run_simple02.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Selected Device is: 

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2283981.v-qsvr-1           ...ub-singleuser u182761         00:01:01 R jupyterhub     
2284056.v-qsvr-1           run_simple02.sh  u182761                0 Q batch          

Waiting for Output ████████████████████████████████████████████████████████████

TimeOut 60 seconds: Job is still queued for execution, check for output file later (run_simple02.sh.o2284056)

 Done⬇

########################################################################
#      Date:           Mon 24 Apr 2023 01:15:02 PM PDT
#    Job ID:           2284056.v-qsvr-1.aidevcloud
#      User:           u182761
# Resources:           cput=75:00:00,neednodes=1:gen9:ppn=2,nodes=1:gen9:ppn=2,walltime=06:00:00
########################################################################

##